# Premier essai BM25 pour requêter les DPEFs et chercher des sujets spécifiques.

- Avec bag of words, courts fragemnts privilégiés
- nécessité de ne garder que les phrase avec sujet + verbe + compléments ici
- et d'aller à la maille de la phrase
- embeddings vraiment snécéssaire égalements

On peut voir que certains sujets cocernenntpus certaines entreprises.

In [60]:
# ! pip install rank_bm25 
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi
pd.options.display.max_colwidth=160

# text
from spacy.lang.fr import French
from spacy.cli.download import download
import spacy
# download('fr_core_news_sm')
nlp = French()
def tokenize(input_str):
    return [token.text for token in nlp(input_str)]

In [83]:
dpef_entreprises = "../../data/processed/DPEFs/dpef_paragraphs.csv"
df = pd.read_csv(dpef_entreprises, sep=";")
print(df.shape)
df = df[df.paragraph.notna()]
print(df.shape)
df.head()
df["paragraph_"] = df["paragraph"].apply(tokenize)
df= df.reset_index()

bm25_index = BM25Okapi(df["paragraph_"].values.tolist())

def search(search_string, num_results=10):
    search_tokens = tokenize(search_string)
    scores = bm25_index.get_scores(search_tokens)
    top_indexes = np.argsort(scores)[::-1][:num_results]
    top_scores = scores[top_indexes]
    return top_indexes, top_scores

indexes, scores = search('pêche poisson biodiversité')

(15135, 10)
(13939, 10)


In [84]:
scores

array([15.20710433, 11.26896059,  9.92932259,  9.76904939,  9.61386807,
        9.30389025,  9.17655923,  9.17655923,  8.95955613,  8.9064717 ])

In [85]:
df.loc[indexes, ["denomination","paragraph"]].values

array([['Carrefour',
        '● Carrefour France a mis en rayon quinze références de thon en conserve pêché à la canne ; cette technique de pêche sélective étant moins impactante que le dispositif de concentration de poisson.'],
       ['Carrefour',
        '2.2.3 Développer l’agroécologie 2.2.5 Agir pour une pêche et une aquaculture responsables 2.2.7 Agir pour protéger les forêts 2.3.4 Développer l’apiculture 2.3.5 Construire et rénover durablement, dans le respect de la biodiversité'],
       ['Carrefour', 'la pêche et'],
       ['Carrefour', 'l’offre de pêche'],
       ['Carrefour', 'la pêche durable, en'],
       ['Carrefour',
        '● privilégier les espèces abondantes, les produits certifiés MSC pêche durable et les techniques de pêche aux impacts moindres sur les écosystèmes. Carrefour commercialisation des espèces sensibles ;'],
       ['Carrefour', 'la pêche électrique. Carrefour France a ainsi'],
       ['Carrefour',
        'Agir pour une pêche et une aquaculture responsa